In [2]:
!pip install --upgrade datasets fsspec huggingface_hub

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 24.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 17.6 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
  Attempting uninstall: datasets
    Found existing installation: datasets 2.14.4
    Uninstalling datasets-2.14.4:
      Successfully uninstalled datasets-2.14.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2025.3.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; pl

In [3]:
from datasets import load_dataset

ds = load_dataset("Teklia/IAM-line")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/2.14k [00:00<?, ?B/s]

train.parquet:   0%|          | 0.00/167M [00:00<?, ?B/s]

validation.parquet:   0%|          | 0.00/24.7M [00:00<?, ?B/s]

test.parquet:   0%|          | 0.00/73.6M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/6482 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/976 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/2915 [00:00<?, ? examples/s]

In [4]:
from datasets import DatasetDict

print(ds)

print(ds['train'][0])

DatasetDict({
    train: Dataset({
        features: ['image', 'text'],
        num_rows: 6482
    })
    validation: Dataset({
        features: ['image', 'text'],
        num_rows: 976
    })
    test: Dataset({
        features: ['image', 'text'],
        num_rows: 2915
    })
})
{'image': <PIL.JpegImagePlugin.JpegImageFile image mode=L size=2467x128 at 0x7EE0C1F7FA90>, 'text': 'put down a resolution on the subject'}


In [5]:
from transformers import TrOCRProcessor

processor = TrOCRProcessor.from_pretrained("microsoft/trocr-base-handwritten")


preprocessor_config.json:   0%|          | 0.00/224 [00:00<?, ?B/s]

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


tokenizer_config.json:   0%|          | 0.00/1.12k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/772 [00:00<?, ?B/s]

In [6]:
def preprocess(example):
    image = example["image"].convert("RGB")

    pixel_values = processor(images=image, return_tensors="pt").pixel_values[0]

    labels = processor.tokenizer(
        example["text"],
        padding="max_length",
        truncation=True,
        max_length=128
    ).input_ids

    labels = [label if label != processor.tokenizer.pad_token_id else -100 for label in labels]

    return {"pixel_values": pixel_values, "labels": labels}

In [7]:
ds = ds.map(preprocess, remove_columns=ds["train"].column_names)


Map:   0%|          | 0/6482 [00:00<?, ? examples/s]

Map:   0%|          | 0/976 [00:00<?, ? examples/s]

Map:   0%|          | 0/2915 [00:00<?, ? examples/s]

In [8]:
from torch.utils.data import Dataset

class IAMDataset(Dataset):
    def __init__(self, hf_dataset):
        self.dataset = hf_dataset

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        item = self.dataset[idx]
        return {
            "pixel_values": item["pixel_values"],
            "labels": item["labels"]
        }


In [9]:
from torch.utils.data import DataLoader

train_dataset = IAMDataset(ds["train"])
val_dataset = IAMDataset(ds["validation"])

train_dataloader = DataLoader(train_dataset, batch_size=4, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=4)


In [10]:
from transformers import VisionEncoderDecoderModel

model = VisionEncoderDecoderModel.from_pretrained("microsoft/trocr-base-handwritten")
model.config.decoder_start_token_id = processor.tokenizer.cls_token_id
model.config.pad_token_id = processor.tokenizer.pad_token_id
model.config.vocab_size = model.decoder.config.vocab_size


config.json:   0%|          | 0.00/4.17k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.33G [00:00<?, ?B/s]

Config of the encoder: <class 'transformers.models.vit.modeling_vit.ViTModel'> is overwritten by shared encoder config: ViTConfig {
  "attention_probs_dropout_prob": 0.0,
  "encoder_stride": 16,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.0,
  "hidden_size": 768,
  "image_size": 384,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "model_type": "vit",
  "num_attention_heads": 12,
  "num_channels": 3,
  "num_hidden_layers": 12,
  "patch_size": 16,
  "pooler_act": "tanh",
  "pooler_output_size": 768,
  "qkv_bias": false,
  "torch_dtype": "float32",
  "transformers_version": "4.51.3"
}

Config of the decoder: <class 'transformers.models.trocr.modeling_trocr.TrOCRForCausalLM'> is overwritten by shared decoder config: TrOCRConfig {
  "activation_dropout": 0.0,
  "activation_function": "gelu",
  "add_cross_attention": true,
  "attention_dropout": 0.0,
  "bos_token_id": 0,
  "classifier_dropout": 0.0,
  "cross_attention_hidden_size": 768,
  "d_mod

generation_config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [11]:
from torch.cuda.amp import autocast, GradScaler
scaler = GradScaler()


<ipython-input-11-91324fc5889e>:2: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = GradScaler()


In [12]:
from torch.optim import AdamW

optimizer = AdamW(model.parameters(), lr=5e-5)

In [13]:
def collate_fn(batch):
    pixel_values = torch.stack([x["pixel_values"] for x in batch])
    labels = [torch.tensor(x["labels"]) for x in batch]

    labels = torch.nn.utils.rnn.pad_sequence(labels, batch_first=True, padding_value=processor.tokenizer.pad_token_id)

    return {"pixel_values": pixel_values, "labels": labels}


In [ ]:
import torch
from tqdm import tqdm
from torch.utils.data import DataLoader, Subset

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

def collate_fn(batch):
    pixel_values = torch.stack([torch.as_tensor(x["pixel_values"]) for x in batch])
    labels = [torch.tensor(x["labels"]) for x in batch]

    labels = torch.nn.utils.rnn.pad_sequence(
        labels, batch_first=True, padding_value=processor.tokenizer.pad_token_id
    )

    return {"pixel_values": pixel_values, "labels": labels}

train_dataset_subset = Subset(train_dataset, range(10))
val_dataset_subset = Subset(val_dataset, range(3))

train_dataloader = DataLoader(
    train_dataset_subset, batch_size=4, shuffle=True,
    collate_fn=collate_fn, num_workers=0, pin_memory=True
)
val_dataloader = DataLoader(
    val_dataset_subset, batch_size=1, shuffle=False,
    collate_fn=collate_fn, num_workers=0, pin_memory=True
)

for epoch in range(1):
    model.train()
    loop = tqdm(train_dataloader, leave=True)
    total_loss = 0

    for batch in loop:
        pixel_values = batch["pixel_values"].to(device)
        labels = batch["labels"].to(device)

        optimizer.zero_grad()
        with autocast():
            outputs = model(pixel_values=pixel_values, labels=labels)
            loss = outputs.loss

        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

        loop.set_description(f"Epoch {epoch}")
        loop.set_postfix(loss=loss.item())
        total_loss += loss.item()

    avg_loss = total_loss / len(train_dataloader)
    print(f"Epoch {epoch} Avg Loss: {avg_loss:.4f}")

model.eval()
predictions = []
references = []

for batch in tqdm(val_dataloader):
    pixel_values = batch["pixel_values"].to(device)
    labels = batch["labels"]

    with torch.no_grad():
        generated_ids = model.generate(pixel_values.to(device))

    pred_texts = processor.batch_decode(generated_ids, skip_special_tokens=True)

    cleaned_labels = []
    for label in labels:
        label = label[label != -100]
        label = label[label < processor.tokenizer.vocab_size]
        cleaned_labels.append(label)

    label_texts = processor.batch_decode(cleaned_labels, skip_special_tokens=True)

    predictions.extend(pred_texts)
    references.extend(label_texts)

for i, (pred, ref) in enumerate(zip(predictions, references)):
    print(f"\nSample {i+1}:")
    print(f"Prediction: {pred}")
    print(f"Reference : {ref}")


  0%|          | 0/3 [00:00<?, ?it/s]<ipython-input-17-83f4d3ec765b>:44: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():
Epoch 0: 100%|██████████| 3/3 [02:17<00:00, 45.84s/it, loss=5.41]


Epoch 0 Avg Loss: 6.5096


100%|██████████| 3/3 [00:57<00:00, 19.16s/it]


Sample 1:
Prediction: A K is a Mr Gakell M for Mr Gakell M for Mr Gakell M for Mr
Reference : It was a splendid interpretation of the

Sample 2:
Prediction: sympathetic to Mr Gakekekekekekekekekekekekekeke
Reference : sympathetic C O . Paul Daneman gave another

Sample 3:
Prediction: past of Mr Ga to Mr Gakell P to Mr Gakell Mr Gakell Mr Gakell from
Reference : part . The rest of the cast were well chosen ,


In [2]:
!pip install evaluate
import evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 7.9 MB/s eta 0:00:00


In [ ]:
cer_metric = evaluate.load("cer")

cer = cer_metric.compute(predictions=predictions, references=references)
print(f"Character Error Rate (CER): {cer:.4f}")
wer_metric = evaluate.load("wer")

wer = wer_metric.compute(predictions=predictions, references=references)
print(f"Word Error Rate (WER): {wer:.4f}")


In [15]:
import torch
from tqdm import tqdm
from torch.utils.data import DataLoader

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

def collate_fn(batch):
    pixel_values = torch.stack([torch.as_tensor(x["pixel_values"]) for x in batch])
    labels = [torch.tensor(x["labels"]) for x in batch]


    labels = torch.nn.utils.rnn.pad_sequence(labels, batch_first=True, padding_value=processor.tokenizer.pad_token_id)


    return {"pixel_values": pixel_values, "labels": labels}

train_dataloader = DataLoader(train_dataset, batch_size=4, shuffle=True, collate_fn=collate_fn)
val_dataloader = DataLoader(val_dataset, batch_size=4, collate_fn=collate_fn)


for epoch in range(1):
    model.train()
    loop = tqdm(train_dataloader, leave=True)
    total_loss = 0

    for batch in loop:
        pixel_values = batch["pixel_values"].to(device)
        labels = batch["labels"].to(device)

        optimizer.zero_grad()
        with autocast():
            outputs = model(pixel_values=pixel_values, labels=labels)
            loss = outputs.loss

        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

        loop.set_description(f"Epoch {epoch}")
        loop.set_postfix(loss=loss.item())
        total_loss += loss.item()

    avg_loss = total_loss / len(train_dataloader)
    print(f"Epoch {epoch} Avg Loss: {avg_loss:.4f}")

  0%|          | 0/1621 [00:00<?, ?it/s]<ipython-input-15-26af81575133>:38: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():
`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.
Epoch 0:  79%|███████▉  | 1287/1621 [33:41<08:55,  1.60s/it, loss=3.48]Exception ignored in: <function _xla_gc_callback at 0x7edf9e83dda0>
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/jax/_src/lib/__init__.py", line 96, in _xla_gc_callback
    def _xla_gc_callback(*args):
    
KeyboardInterrupt: 
Epoch 0:  80%|███████▉  | 1292/1621 [33:49<08:10,  1.49s/it, loss=3.22]Exception ignored in: <function _xla_gc_callback at 0x7edf9e83dda0>
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/jax/_src/lib/__init__.py", line 96, in _xla_gc_callback
    def _xla_gc_callback(*args):
    
KeyboardInterrupt: 
Epoch

KeyboardInterrupt: 